<a href="https://colab.research.google.com/github/frnkly/jayne-eats/blob/main/ml/notes/model-exploration-tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Exploration

- [Choosing the right estimator](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [Tuning the hyper-parameters of an estimator using grid search](https://scikit-learn.org/stable/modules/grid_search.html)



# Data

In [5]:
# Data samples, organized as: [classification, consistency, temperature, [ingredients]].
foods = [
  # Pizzas
  [0, 'solid', 'warm', ['bell pepper', 'cheese', 'flour', 'garlic', 'oil', 'olive', 'onion', 'salt', 'sugar', 'tomato']], # Olives
  [0, 'solid', 'warm', ['anchovy', 'bell pepper', 'cheese', 'flour', 'garlic', 'mayonnaise', 'oil', 'olive', 'onion', 'salt', 'spinach', 'sugar']], # Anchovies
  [0, 'solid', 'warm', ['apple', 'coriander', 'cheese', 'flour', 'garlic', 'oil', 'onion', 'salt', 'sugar']], # Apple & goat cheese
  [1, 'solid', 'warm', ['arugula', 'bell pepper', 'cheese', 'coriander', 'flour', 'garlic', 'oil', 'onion', 'salt', 'spinach', 'sugar', 'tomato']], # Arugula
  [1, 'solid', 'warm', ['bell pepper', 'cheese', 'flour', 'garlic', 'mushroom', 'oil', 'onion', 'salt', 'sugar']], # Mushroom
  [1, 'solid', 'warm', ['cheese', 'flour', 'garlic', 'oil', 'onion', 'salt', 'sugar', 'tomato']], # Cheese

  # Products
  # [1, 'crunchy', 'normal', ['anise', 'baking powder', 'egg', 'flour', 'oil', 'sugar']], # Biscotti
  # [1, 'crunchy', 'cool', ['almond', 'barley malt', 'milk', 'rice', 'salt', 'sugar', 'wheat bran', 'whole grain wheat']], # Vanilla almond Special K

  # Smoothies & milkshakes
  [1, 'chewy', 'cool', ['cookies', 'milk', 'skim milk', 'sugar', 'vanilla']], # Oreo ice cream
  [0, 'thick', 'cool', ['blueberry', 'pineapple', 'strawberry']],
  [0, 'thick', 'cool', ['blueberry', 'kiwi', 'strawberry']],
  [0, 'thick', 'cool', ['chocolate', 'milk', 'vanilla']],
  [0, 'thick', 'cool', ['banana', 'chocolate', 'milk', 'vanilla']],
  [0, 'thick', 'cool', ['milk', 'strawberry', 'vanilla']],
  [1, 'thick', 'cool', ['banana', 'mango', 'pineapple']],
  [1, 'thick', 'cool', ['blueberry', 'kiwi', 'mango', 'pineapple']],
  [1, 'thick', 'cool', ['banana', 'milk', 'strawberry', 'vanilla']],
  [1, 'thick', 'cool', ['banana', 'milk', 'kiwi', 'vanilla']],

  # Soups & chilis
  [0, 'mashed', 'warm', ['bay leaf', 'chickpea', 'chili', 'coriander', 'cumin', 'garlic', 'ginger', 'oil', 'onion', 'salt', 'tomato', 'tumeric']], # Chole chickpea curry
  [0, 'mashed', 'warm', ['black pepper', 'chickpea', 'chili', 'coriander', 'cumin', 'garlic', 'ginger', 'oil', 'onion', 'potato', 'salt', 'spinach', 'tomato', 'tumeric']], # Sweet potato curry
  [1, 'mashed', 'warm', ['black pepper', 'butter', 'cheese', 'coriander', 'garlic', 'potato', 'salt']], # Mash potatoes
  [1, 'mashed', 'warm', ['bean', 'black pepper', 'carrot', 'chili', 'garlic', 'onion', 'potato', 'salt', 'tomato', 'tumeric']], # Chili
  [0, 'liquid', 'warm', ['black pepper', 'butter', 'flour', 'garlic', 'onion', 'peanut', 'salt']], # Peanut soup
  [1, 'thick', 'warm', ['black pepper', 'carrot', 'chili', 'garlic', 'onion', 'potato', 'salt', 'squash']], # Butternut squash soup

  # Miscellaneous (stuff made out of ingredients from above)
  [1, 'crunchy', 'normal', ['almond', 'apple', 'arugula', 'cranberry', 'oil', 'salt']], # Arugula salad
  [1, 'solid', 'warm', ['black pepper', 'cheese', 'egg', 'garlic', 'mushroom', 'oil', 'onion', 'red pepper', 'salt', 'spinach']], # Mushroom spinach omelet
  [1, 'solid', 'warm', ['bean', 'black pepper', 'oil', 'onion', 'rice', 'salt', 'tumeric']], # Rice and beans
  [1, 'solid', 'warm', ['black pepper', 'mushroom', 'oil', 'onion', 'pea', 'rice', 'salt']], # Riz djondjon
  [1, 'solid', 'cool', ['banana', 'butter', 'egg', 'flour', 'milk', 'salt', 'strawberry']], # Crepes
  [0, 'solid', 'warm', ['black pepper', 'oil', 'okra', 'onion', 'rice', 'salt', 'spinach']], # Kalalou
]

print('Size of data set:', len(foods))

Size of data set: 28


In [6]:
# General utility functions.
import numpy as np

def get_descriptions(samples):
  """Retrieves food descriptions from a data set."""

  return list(map(lambda sample: sample[1:3], foods))

def get_ingredients(samples):
  """Retrieves ingredients from a data set."""

  return list(map(lambda sample: sample[3], foods))

def encode_samples(samples):
  """Encodes food samples to use as inputs to a model."""
  sample_descriptions = list(map(lambda sample: sample[1:3], samples))
  sample_ingredients = list(map(lambda sample: sample[3], samples))
  encoded_data_set = []
  encoded_descriptions = food_descriptions_encoder\
    .transform(sample_descriptions)\
    .toarray()
  encoded_ingredients = ingredients_encoder.transform(sample_ingredients)

  for i in range(len(samples)):
    encoded_data_set.append(np.concatenate([encoded_descriptions[i], encoded_ingredients[i]]))
  
  return encoded_data_set

In [7]:
# Encoders.
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, OneHotEncoder

food_descriptions_encoder = OneHotEncoder()
food_descriptions_encoder.fit(get_descriptions(foods))
ingredients_encoder = MultiLabelBinarizer()
ingredients_encoder.fit(get_ingredients(foods))

print(f'Food descriptions:', food_descriptions_encoder.categories_)
print(f'Ingredients ({len(ingredients_encoder.classes_)}):', ingredients_encoder.classes_)

Food descriptions: [array(['chewy', 'crunchy', 'liquid', 'mashed', 'solid', 'thick'],
      dtype=object), array(['cool', 'normal', 'warm'], dtype=object)]
Ingredients (48): ['almond' 'anchovy' 'apple' 'arugula' 'banana' 'bay leaf' 'bean'
 'bell pepper' 'black pepper' 'blueberry' 'butter' 'carrot' 'cheese'
 'chickpea' 'chili' 'chocolate' 'cookies' 'coriander' 'cranberry' 'cumin'
 'egg' 'flour' 'garlic' 'ginger' 'kiwi' 'mango' 'mayonnaise' 'milk'
 'mushroom' 'oil' 'okra' 'olive' 'onion' 'pea' 'peanut' 'pineapple'
 'potato' 'red pepper' 'rice' 'salt' 'skim milk' 'spinach' 'squash'
 'strawberry' 'sugar' 'tomato' 'tumeric' 'vanilla']


In [8]:
# Encode, randomize and split data set.
from sklearn.model_selection import train_test_split

X = encode_samples(foods)
y = list(map(lambda sample: sample[0], foods))
X_train, X_test, y_train, y_test = train_test_split(X, y)

print('Training set size:', len(y_train))
print('Testing set size:', len(y_test))

Training set size: 21
Testing set size: 7


In [9]:
# Data-related utility functions.
from sklearn.model_selection import cross_val_score

def train(model_class, highlight=False, **args):
  """Trains a Scikit-Learn-based model."""
  model = model_class(**args)
  cross_validation = cross_val_score(model, X_train, y_train, cv=4)
  cv_mean_score = round(cross_validation.mean(), 3)
  training_set_score = round(model.fit(X_train, y_train).score(X_test, y_test), 3)

  arg_str = ', '.join([f'{key}={args[key]}' for key in args])
  name = f'{model_class.__name__}({arg_str})'
  output = f'{name.ljust(40, ".")} {cv_mean_score}, {training_set_score}'

  if (highlight):
    output += ' **'

  print(output)

  return None

# [Nearest Neighbours](https://scikit-learn.org/stable/modules/neighbors.html)

Scikit-learn supports three nearest neighbours algorithms:
- [Brute-force](https://scikit-learn.org/stable/modules/neighbors.html#brute-force).
- [K-D Tree](https://scikit-learn.org/stable/modules/neighbors.html#k-d-tree): best for low dimensional data (D < 20).
- [Ball Tree](https://scikit-learn.org/stable/modules/neighbors.html#ball-tree): better for high dimensional data.

## Summary

- Higher K is better.
- Uniform weights might be slightly better than distance weights.
- Top performers: Uniform KNN (K=15)

In [10]:
from sklearn.neighbors import KNeighborsClassifier as KNN

train(KNN, n_neighbors=3, weights='distance')
train(KNN, n_neighbors=15, weights='distance')
train(KNN, n_neighbors=1, weights='uniform')
train(KNN, n_neighbors=3, weights='uniform')
train(KNN, n_neighbors=7, weights='uniform')
train(KNN, n_neighbors=15, weights='uniform', highlight=True)
# train(KNN, n_neighbors=20, weights='uniform')

KNeighborsClassifier(n_neighbors=3, weights=distance) 0.292, 0.429
KNeighborsClassifier(n_neighbors=15, weights=distance) 0.475, 0.286
KNeighborsClassifier(n_neighbors=1, weights=uniform) 0.392, 0.429
KNeighborsClassifier(n_neighbors=3, weights=uniform) 0.292, 0.429
KNeighborsClassifier(n_neighbors=7, weights=uniform) 0.433, 0.571
KNeighborsClassifier(n_neighbors=15, weights=uniform) 0.525, 0.286 **


# [Random Forest](https://scikit-learn.org/stable/modules/ensemble.html#forests-of-randomized-trees)

TODO

# [Stochastic Gradient Descent](https://scikit-learn.org/stable/modules/sgd.html)

Requires a lot of data (>100k)

# [Support Vector Machine](https://scikit-learn.org/stable/modules/svm.html)

## Summary

- Higher polynomial is better.
- Lower C is better for RBF.
- Linear doesn't work very well.
- Top performers:
  1. SVC(C=10.0, gamma=0.63, kernel=rbf)
  2. SVC(C=4.0, gamma=1.0, kernel=rbf)
  3. SVC(C=10.0, gamma=1.22, kernel=rbf)
  4. SVC(degree=10, kernel=poly)
  5. SVC(kernel=sigmoid)

In [11]:
# SVC: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
from sklearn.svm import SVC

# Linear
train(SVC, kernel='linear')

# Polynomial
train(SVC, degree=3, kernel='poly')
train(SVC, degree=10, kernel='poly', highlight=True)
train(SVC, degree=20, kernel='poly')

# Radial basis function
train(SVC, C=0.01, kernel='rbf')
train(SVC, C=0.1, kernel='rbf')
train(SVC, C=0.1, gamma=0.1, kernel='rbf')
train(SVC, C=0.1, gamma=1.0, kernel='rbf')
train(SVC, C=0.1, gamma=10.0, kernel='rbf')
train(SVC, C=1.0, kernel='rbf')
train(SVC, C=4.0, kernel='rbf')
train(SVC, C=4.0, gamma=0.1, kernel='rbf')
train(SVC, C=4.0, gamma=0.11, kernel='rbf', highlight=True)
train(SVC, C=4.0, gamma=0.12, kernel='rbf')
train(SVC, C=4.0, gamma=0.15, kernel='rbf')
train(SVC, C=4.0, gamma=0.2, kernel='rbf')
train(SVC, C=4.0, gamma=0.5, kernel='rbf')
train(SVC, C=4.0, gamma=1.0, kernel='rbf', highlight=True)
train(SVC, C=4.0, gamma=2.0, kernel='rbf')
train(SVC, C=4.0, gamma=3.0, kernel='rbf')
train(SVC, C=4.0, gamma=4.0, kernel='rbf', highlight=True)
train(SVC, C=4.0, gamma=5.0, kernel='rbf')
train(SVC, C=4.0, gamma=10.0, kernel='rbf')
train(SVC, C=5.0, kernel='rbf')
train(SVC, C=8.0, kernel='rbf')
train(SVC, C=10.0, kernel='rbf')
train(SVC, C=10.0, gamma=0.1, kernel='rbf')
train(SVC, C=10.0, gamma=0.63, kernel='rbf', highlight=True)
train(SVC, C=10.0, gamma=1.0, kernel='rbf')
train(SVC, C=10.0, gamma=1.22, kernel='rbf', highlight=True)
train(SVC, C=10.0, gamma=4.0, kernel='rbf', highlight=True)
train(SVC, C=10.0, gamma=10.0, kernel='rbf')

# Sigmoid
train(SVC, kernel='sigmoid', highlight=True)


SVC(kernel=linear)...................... 0.383, 0.429
SVC(degree=3, kernel=poly).............. 0.425, 0.571
SVC(degree=10, kernel=poly)............. 0.575, 0.571 **
SVC(degree=20, kernel=poly)............. 0.575, 0.714
SVC(C=0.01, kernel=rbf)................. 0.425, 0.714
SVC(C=0.1, kernel=rbf).................. 0.425, 0.714
SVC(C=0.1, gamma=0.1, kernel=rbf)....... 0.425, 0.714
SVC(C=0.1, gamma=1.0, kernel=rbf)....... 0.475, 0.714
SVC(C=0.1, gamma=10.0, kernel=rbf)...... 0.475, 0.714
SVC(C=1.0, kernel=rbf).................. 0.375, 0.571
SVC(C=4.0, kernel=rbf).................. 0.342, 0.571
SVC(C=4.0, gamma=0.1, kernel=rbf)....... 0.342, 0.571
SVC(C=4.0, gamma=0.11, kernel=rbf)...... 0.342, 0.571 **
SVC(C=4.0, gamma=0.12, kernel=rbf)...... 0.342, 0.571
SVC(C=4.0, gamma=0.15, kernel=rbf)...... 0.383, 0.571
SVC(C=4.0, gamma=0.2, kernel=rbf)....... 0.383, 0.571
SVC(C=4.0, gamma=0.5, kernel=rbf)....... 0.475, 1.0
SVC(C=4.0, gamma=1.0, kernel=rbf)....... 0.475, 1.0 **
SVC(C=4.0, gamma=2.0, k

# [SVC Parameter Tuning](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html)

Some [scoring functions](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter) explained:

- [Accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)
- [Average precision](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html)
- [Recall](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html)

Glossary:

- [Precision](https://developers.google.com/machine-learning/glossary#precision): rate at which model is correct (kind of like "engineering precision").
- [Recall](https://developers.google.com/machine-learning/glossary#recall): rate at which model can make successful predictions (kind of like "engineering accuracy").

In [12]:
# from sklearn.metrics import classification_report
# from sklearn.model_selection import GridSearchCV

# parameters = [
#   {
#     'C': range(4, 31),
#     'gamma': list(map(lambda g: g / 100, list(range(10, 1001)))),
#     'kernel': ['rbf'],
#   }
# ]

# score_fns = [
#   # 'accuracy',
#   'recall_macro',
# ]

# for score_fn in score_fns:
#     print(f'Tuning hyper-parameters for {score_fn}...')
#     print()

#     model = GridSearchCV(SVC(), parameters, scoring=score_fn)
#     model.fit(X_train, y_train)

#     print(f'Best parameters set for {score_fn}')
#     print()
#     print(model.best_params_)
#     print()

#     print('Detailed classification report:')
#     print()
  
#     y_true, y_pred = y_test, model.predict(X_test)
#     print(classification_report(y_true, y_pred))
#     print()

# TensorFlow [Linear Classifier](https://www.tensorflow.org/tutorials/estimator/linear)


In [22]:
# Linear classifier using TensorFlow
import pandas as pd
import tensorflow as tf

features = []

for category in food_descriptions_encoder.categories_:
  features.extend(category)

features.extend(ingredients_encoder.classes_)

X_train_df = pd.DataFrame(X_train, columns=features)
X_eval_df = pd.DataFrame(X_test, columns=features)
y_train_df = pd.DataFrame(y_train, columns=['label'])
y_eval_df = pd.DataFrame(y_test, columns=['label'])

X_train_df.head()

,chewy,crunchy,liquid,mashed,solid,thick,cool,normal,warm,almond,anchovy,apple,arugula,banana,bay leaf,bean,bell pepper,black pepper,blueberry,butter,carrot,cheese,chickpea,chili,chocolate,cookies,coriander,cranberry,cumin,egg,flour,garlic,ginger,kiwi,mango,mayonnaise,milk,mushroom,oil,okra,olive,onion,pea,peanut,pineapple,potato,red pepper,rice,salt,skim milk,spinach,squash,strawberry,sugar,tomato,tumeric,vanilla
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [23]:
X_train_df.describe()

,chewy,crunchy,liquid,mashed,solid,thick,cool,normal,warm,almond,anchovy,apple,arugula,banana,bay leaf,bean,bell pepper,black pepper,blueberry,butter,carrot,cheese,chickpea,chili,chocolate,cookies,coriander,cranberry,cumin,egg,flour,garlic,ginger,kiwi,mango,mayonnaise,milk,mushroom,oil,okra,olive,onion,pea,peanut,pineapple,potato,red pepper,rice,salt,skim milk,spinach,squash,strawberry,sugar,tomato,tumeric,vanilla
count,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.0,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.0,21.000000,21.000000,21.000000,21.000000,21.000000
mean,0.047619,0.047619,0.047619,0.142857,0.428571,0.285714,0.380952,0.047619,0.571429,0.047619,0.047619,0.095238,0.047619,0.190476,0.047619,0.095238,0.142857,0.285714,0.095238,0.095238,0.047619,0.285714,0.095238,0.142857,0.095238,0.047619,0.142857,0.047619,0.095238,0.095238,0.333333,0.476190,0.095238,0.095238,0.095238,0.047619,0.238095,0.095238,0.523810,0.047619,0.095238,0.571429,0.0,0.047619,0.142857,0.095238,0.047619,0.095238,0.666667,0.047619,0.190476,0.0,0.095238,0.285714,0.238095,0.190476,0.190476
std,0.218218,0.218218,0.218218,0.358569,0.507093,0.462910,0.497613,0.218218,0.507093,0.218218,0.218218,0.300793,0.218218,0.402374,0.218218,0.300793,0.358569,0.462910,0.300793,0.300793,0.218218,0.462910,0.300793,0.358569,0.300793,0.218218,0.358569,0.218218,0.300793,0.300793,0.483046,0.511766,0.300793,0.300793,0.300793,0.218218,0.436436,0.300793,0.511766,0.218218,0.300793,0.507093,0.0,0.218218,0.358569,0.300793,0.218218,0.300793,0.483046,0.218218,0.402374,0.0,0.300793,0.462910,0.436436,0.402374,0.402374
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,1.000000,

In [25]:
# Feature columns
tf_columns = []

for feature in features:
  tf_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature, [0, 1]))

In [26]:
# Input functions

def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    return ds
  
  return input_function

train_input_fn = make_input_fn(X_train_df, y_train_df)
eval_input_fn = make_input_fn(X_eval_df, y_eval_df, num_epochs=1, shuffle=False)


In [28]:
# Linear estimator
linear_est = tf.estimator.LinearClassifier(feature_columns=tf_columns)
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

print(result)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpuyl0t3h9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initiali

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


ValueError: ignored